## x.1 数据集介绍

最常见的数据集有98年手写数字的MNIST数据集，有60,000张28*28分辨率图像(加上10,000张测试图像)，但大部分模型在这个数据集上的分类准确率都高于95%；

09年ImageNet数据集，数据量太过于庞大；

推荐17年Fashion-MNIST数据集，包含10类28*28像素的服饰图像；

In [1]:
%matplotlib inline
import time
import torch
import torchvision
from torchvision import transforms
import core

core.use_svg_display()

由于Fashion-MNIST数据集很常用，torchvision也提供了它的原始数据集和预处理版本，我们在这里直接使用torchvision导入数据集。注意使用torchvision返回的也是**Dataset的实例对象**。

我们还使用了`torchvision.transforms.Compose`中torchvision的强大预处理工具

In [4]:
class FashionMNIST(core.DataModule):
    def __init__(self, batch_size=64, resize=(28, 28), root="/home/yingmuzhi/_learning/d2l/data"):
        super().__init__()
        self.save_hyperparameters()
        trans = torchvision.transforms.Compose([transforms.Resize(resize),
                                                transforms.ToTensor()])
        self.train = torchvision.datasets.FashionMNIST(
            root=self.root, train=True, transform=trans, download=False)
        self.val = torchvision.datasets.FashionMNIST(
            root=self.root, train=False, transform=trans, download=False)

我们看看train和validation数据集中有多少数据

In [5]:
data = FashionMNIST(resize=(32, 32))
len(data.train), len(data.val)

(60000, 10000)

大部分的现代图形有三个通道（RGB）或者四通道（RGBA），高光谱图像可能有超过100个通道（HyMap）。但无所谓，按照惯例我们将图像存储为`c*h*w`张量，分别对应channel, height, width。我们查看图像的多维信息。

通过下面代码我们能够发现在Dataset实例对象中，每次返回的是一个tuple，对应（signal, target），而signal和target都是Tensor类型。

在下面代码的最后一行，我们打印出每一个图像Tensor的shape信息，如下，

In [12]:
print(type(data.train))
print(len(data.train))
print(len(data.train[0]))
print(type(data.train[0][0]))
print(data.train[0][0].shape)

<class 'torchvision.datasets.mnist.FashionMNIST'>
60000
2
<class 'torch.Tensor'>
torch.Size([1, 32, 32])


我们查看target标签如下

In [13]:
@core.add_to_class(FashionMNIST)
def text_labels(self, indices):
    """Return text labels."""
    labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
              'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [labels[int(i)] for i in indices]

我们已经有train和val的Dataset了，接下来我们只需要重写`get_dataloader`方法，使其根据传入的train的flag读取train的Dataset或者val的Dataset。

In [14]:
@core.add_to_class(FashionMNIST)
def get_dataloader(self, train):
    data = self.train if train else self.val
    return torch.utils.data.DataLoader(data, self.batch_size, shuffle=train,
                                       num_workers=self.num_workers)

至此，我们便完成了DataModule部分，即得到了train的DataLoader和val的DataLoader.

我们下面使用经典语法`next(iter(...DataLoader))`来对得到的DataLoader进行测试

通过观察我们能够发现图像的组织形式往往是(Batch_Size, Channel, Slice, Height, Width)，而其中的元素类型则是torch.float32类型。

In [15]:
X, y = next(iter(data.train_dataloader()))
print(X.shape, X.dtype, y.shape, y.dtype)

torch.Size([64, 1, 32, 32]) torch.float32 torch.Size([64]) torch.int64


我们查看使用DataLoader读取数据会多快

In [16]:
tic = time.time()
for X, y in data.train_dataloader():
    continue
f'{time.time() - tic:.2f} sec'

'3.57 sec'

In [ ]:
@core.add_to_class(FashionMNIST)  #@save
def visualize(self, batch, nrows=1, ncols=8, labels=[]):
    X, y = batch
    if not labels:
        labels = self.text_labels(y)
    core.show_images(X.squeeze(1), nrows, ncols, titles=labels)
batch = next(iter(data.val_dataloader()))
data.visualize(batch)